In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
from skimage.draw import polygon
import SimpleITK as sitk
import pydicom as dicom
from pydicom.tag import Tag
import sys
import csv
import pandas as pd
import pydicom as pdcm

In [10]:
window_type = 'lung'
root_path="//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/"
structured = root_path + "DATA_VOLLEDIG_structured/"
unstructured = root_path + "DATA_VOLLEDIG_unstructured/"
save_csv_path = "C:/Users/delaOArevaLR/OneDrive - UMCG/Code/Code_From_Umcg/UMCG_ImageFormattingCode/Dicom2Nifti_Public/"

image_generation_flag=True
mask_generation_flag=True

#Folders to look at:
Try1 ="CT_DICOM_CLEAN/"
Try2 ="PET_DICOM/"
Try3 ="PET_LDCT_REGISTERED/"
Try4 ="PET_MIRADA_REGISTERED/"

In [11]:
PxList = np.sort(os.listdir(structured+Try1))
print("Number of Patients:",len(PxList)," Example: ",PxList[0])

Number of Patients: 664  Example:  0004879


In [12]:
#modality to look for
#PT, CT, RTSTRUCT
#CT is: ACCTBody , Avg Ct, 0-90% Breathing Phases
#PET is PET_Earl
#RTStruct - GTV - ITV
modalityToLook = ["CT","PET"]


In [18]:
Trys = [Try1,Try2,Try3,Try4]
Trys = [unstructured]

In [19]:
#Create 5 CSV 
count=0
for CurrentTry in Trys:
    print("CurentTry: ",count)
    PxList = np.sort(os.listdir(CurrentTry))
    csv_filename = save_csv_path + "FPaths_Try"+str(count)+"_v3.csv"
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Px','AvgCt','RTSTruct','ACCTBody','PET','0%','10%','20%','30%','40%','50%','60%','70%','80%','90%'])
        for Px in PxList:
            ACCT_LD_fpaths = [""]
            PET_Earl_fpaths = [""]
            AvgCt_fpaths = [""]
            BP0_CT_fpaths,BP10_CT_fpaths,BP20_CT_fpaths,BP30_CT_fpaths,BP40_CT_fpaths  = ([""],[""],[""],[""],[""])
            BP50_CT_fpaths,BP60_CT_fpaths,BP70_CT_fpaths,BP80_CT_fpaths,BP90_CT_fpaths = ([""],[""],[""],[""],[""])
            RTStruct_fpaths = [""]
            
            miss_RTStruct = True
            
            for root, dirs, files in os.walk(CurrentTry+Px, topdown=True):
                for d in dirs:
                    if "Thorax" in d or "Ave" in d:
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if 'RTSTRUCT' in d or "pproved" in d:
                        RTStruct_fpaths.append(root+'/'+d+'/')
                        miss_RTStruct = False
                    if "AC_CT_Body" in d or "CT van PET" in d or "CT LD" in d or "AC CT" in d or "AC  CT" in d:
                        ACCT_LD_fpaths.append(root+'/'+d+'/')
                    if "PET" in d or "Earl" in d:
                        PET_Earl_fpaths.append(root+'/'+d+'/')
                    if (" 0%" in d or "=0%" in d or "100%" in d) and not"MIP" in d: BP0_CT_fpaths.append(root+'/'+d+'/')
                    if "10%" in d and not "MIP" in d: BP10_CT_fpaths.append(root+'/'+d+'/')
                    if "20%" in d and not "MIP" in d: BP20_CT_fpaths.append(root+'/'+d+'/')
                    if "30%" in d and not "MIP" in d: BP30_CT_fpaths.append(root+'/'+d+'/')
                    if "40%" in d and not "MIP" in d: BP40_CT_fpaths.append(root+'/'+d+'/')
                    if "50%" in d and not "MIP" in d: BP50_CT_fpaths.append(root+'/'+d+'/')
                    if "60%" in d and not "MIP" in d: BP60_CT_fpaths.append(root+'/'+d+'/')
                    if "70%" in d and not "MIP" in d: BP70_CT_fpaths.append(root+'/'+d+'/')
                    if "80%" in d and not "MIP" in d: BP80_CT_fpaths.append(root+'/'+d+'/')
                    if "90%" in d and not "MIP" in d: BP90_CT_fpaths.append(root+'/'+d+'/')
            
            if miss_RTStruct:
                for root, dirs, files in os.walk(CurrentTry+Px, topdown=True):
                    for d in dirs:
                        if "PinnPlan" in d and not("RTDOSE" in d):
                            for roots, dirss, filess in os.walk(root+"/"+d, topdown=True):
                                for f in filess:
                                    try:
                                        slices = [pdcm.read_file(roots+'/'+f)]
                                        modality = slices[0].Modality
                                        if modality =="RTSTRUCT":
                                            RTStruct_fpaths.append(root+'/'+d+'/')
                                            break                                    
                                    except Exception as e:
                                        print(f"An exception occurred: {e}")
                                       
                                    
            
            writer.writerow([Px,AvgCt_fpaths[-1],RTStruct_fpaths[-1],ACCT_LD_fpaths[-1],PET_Earl_fpaths[-1],BP0_CT_fpaths[-1],BP10_CT_fpaths[-1],BP20_CT_fpaths[-1],BP30_CT_fpaths[-1],BP40_CT_fpaths[-1],BP50_CT_fpaths[-1],BP60_CT_fpaths[-1],BP70_CT_fpaths[-1],BP80_CT_fpaths[-1],BP90_CT_fpaths[-1]])
            
    count+=1

CurentTry:  0
An exception occurred: [Errno 2] No such file or directory: '//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/2939330\\1.2.528.1.1008.345049137191.1361877475.14602.2/PinnPlan= Thorax (Thu Apr 04 162503 2013)/2.16.840.1.113669.2.931128.289629209.20130404162523.395952.DCM'


In [28]:
CurrentTry

'//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_structured/PET_DICOM'

In [20]:
file_list = glob.glob(save_csv_path+"/*.csv")
merged_df = pd.DataFrame()
dfs = []
print("fileList:",file_list)
# Iterate over each file
for file in file_list:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    id_column = df.columns[0]
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate the DataFrames vertically
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_data.csv', index=False)


fileList: ['C:/Users/delaOArevaLR/OneDrive - UMCG/Code/Code_From_Umcg/UMCG_ImageFormattingCode/Dicom2Nifti_Public\\FPaths_Try0_v3.csv', 'C:/Users/delaOArevaLR/OneDrive - UMCG/Code/Code_From_Umcg/UMCG_ImageFormattingCode/Dicom2Nifti_Public\\FPaths_Try1_v2.csv']


In [21]:
grouped_df = merged_df.groupby(id_column).agg(lambda x: x.dropna().iloc[0] if not np.all(pd.isnull(x)) else np.nan)

# Reset the index to make the ID column a regular column
grouped_df = grouped_df.reset_index()

In [22]:
 len(grouped_df.columns)

15

In [23]:
# Set the threshold for the number of non-null values required to keep a row
threshold = len(grouped_df.columns) - 15

# Drop rows with more than 13 empty columns
filtered_df = grouped_df.dropna(thresh=threshold)

In [24]:
filtered_df.to_csv(save_csv_path+'/unique_rows_data.csv', index=False)

In [25]:
filtered_df

,Px,AvgCt,RTSTruct,ACCTBody,PET,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%
0,4879,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...
1,10539,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,NaN,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...
2,13194,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...
3,14504,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...
4,17420,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,9933319,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...
830,9935146,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...
831,9936840,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
832,9949900,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...,NaN,NaN,//zkh/appdata/RTDicom/Projectline - Modelling ...


In [26]:
for d in fpaths:
    filesList = []
    filename = d.split('/')[-2]

    for root, dirs, files in os.walk(d, topdown=True):
        for f in files:
            filesList.append(root+f)

    print(root,len(filesList))
    
#    image, pixel_spacing, image_position_patient = convert_dicom_to_nifty(filesList,
#                        patientID,output_folder = save_px_path,labels_rtstruct=structure_for_mask,
#                        extension='.nii.gz',filename=filename)
    

    image, pixel_spacing, image_position_patient,axial_positions = convert_dicom_to_nifty(filesList,
                    patientID,output_folder = save_px_path,labels_rtstruct=structure_for_mask,
                    extension='.nii.gz',filename=filename,pixel_spacing=pixel_spacing,
                    image_position_patient=image_position_patient,axial_positions=axial_positions,
                                                                                  np_image=image)

NameError: name 'fpaths' is not defined

In [ ]:
import warnings
from os.path import join
from datetime import time, datetime
import numpy as np
from skimage.draw import polygon
import SimpleITK as sitk
import pydicom as pdcm
from pydicom.tag import Tag


def convert_dicom_to_nifty(input_filepaths,patientID,
                           output_folder,
                           labels_rtstruct,extension,filename=filename,
                           pixel_spacing = None, image_position_patient=None,axial_positions=None,
                           np_image=None,dtype_image=np.float32,dtype_mask=np.uint8,):
    """Function to convert the dicom files contained in input_filepaths to one
       NIFTI image.

    Args:
        input_filepaths (list): list of the dicom paths
        output_folder (str): path to the output folder where to store the
                             NIFTI file.
        modality (str, optional): The modality of the DICOM, it is used to
                                  obtain the correct physical values
                                  (Hounsfield unit for the CT and SUV for the
                                  PT). Defaults to 'CT'.
        sitk_writer (sitk.WriteImage(), optional): The SimpleITK object used
                                                   to write an array to the
                                                   NIFTI format. Defaults to
                                                   None.
        rtstruct_file (list, optional): Path to the RTSTRUCT file associated
                                       with the current image. Defaults to
                                       None.
        labels_rtstruct (list, optional): List of label to extract from the
                                          RTSTRUCT. Defaults to ['GTVt'].
        patient_weight_from_ct (float, optional): If the patient's weight is
                                                  missing from the PT DICOM
                                                  it can be provided through
                                                  this argument. Defaults to
                                                  None.
        extension (str, optional): The extension in which to save the NIFTI.
                                   Defaults to '.nii'.
        dtype_image (numpy.dtype, optional): The dtype in which to save the
                                             image. Defaults to np.float32.
        dtype_mask (numpy.dtype, optional): The dtype in which to save the
                                            segmentation. Defaults to np.uint8.

    Raises:
        MissingWeightException: Error to alert when the weight is missing from
                                the PT, to compute the SUV.
        RuntimeError: Error to alert when one or more slices are missing
        ValueError: Raised when a modality or a unit (for the PT) is not
                    handled.

    Returns:
        numpy.array: The numpy image, used to compute the bounding boxes
    """

    slices = [pdcm.read_file(dcm) for dcm in input_filepaths]
    modality = slices[0].Modality
    print("working on: ",modality)
    
    if modality != 'RTSTRUCT':
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        if modality == 'PT':
            if float(slices[0][Tag(0x00101030)].value) is None:
                if hasattr(slices[0], 'PatientWeight'):
                    patient_weight = float(slices[0].PatientsWeight)
                elif patient_weight_from_ct is not None:
                    patient_weight = patient_weight_from_ct
                else:
                    raise MissingWeightException(
                        'Cannot compute SUV the weight is missing')
            else:
                patient_weight = float(slices[0][Tag(0x00101030)].value)

        # Check if all the slices come from the same serie
        same_serie_uid = True
        serie_uid = slices[0].SeriesInstanceUID
        for s in slices:
            same_serie_uid *= serie_uid == s.SeriesInstanceUID

        if not same_serie_uid:
            raise RuntimeError('A slice comes from another serie')

        axial_positions = np.asarray([k.ImagePositionPatient[2] for k in slices])
        # Compute redundant slice positions
        ind2rm = [
            ind for ind in range(len(axial_positions))
            if axial_positions[ind] == axial_positions[ind - 1]
        ]
        # Check if there is redundancy in slice positions and remove them
        if len(ind2rm) > 0:
            slices = [k for i, k in enumerate(slices) if i not in ind2rm]
            axial_positions = np.asarray(
                [k.ImagePositionPatient[2] for k in slices])

        slice_spacing = (slices[1].ImagePositionPatient[2] -
                         slices[0].ImagePositionPatient[2])

        pixel_spacing = np.asarray([
            slices[0].PixelSpacing[0],
            slices[0].PixelSpacing[1],
            slice_spacing,
        ])

        if modality == 'CT':
            np_image = get_physical_values_ct(slices, dtype=dtype_image)
        elif modality == 'PT':
            np_image = get_physical_values_pt_new(slices,
                                              patient_weight,
                                              dtype=dtype_image)
        else:
            raise ValueError('The modality {} is not supported'.format(modality))

        position_final_slice = (
            len(slices) - 1) * slice_spacing + slices[0].ImagePositionPatient[2]
        # Test whether some slices are missing
        if not is_approx_equal(position_final_slice,
                               float(slices[-1].ImagePositionPatient[2])):
            if (position_final_slice - axial_positions[-1]) / slice_spacing < 1.5:
                # If only one slice is missing
                diff = np.asarray([
                    not is_approx_equal(
                        float(axial_positions[ind]) -
                        float(axial_positions[ind - 1]) - slice_spacing, 0)
                    for ind in range(1, len(axial_positions))
                ])
                ind2interp = int(np.where(diff)[0])
                new_slice = (np_image[:, :, ind2interp] +
                             np_image[:, :, ind2interp + 1]) * 0.5
                new_slice = new_slice[..., np.newaxis]
                np_image = np.concatenate(
                    (np_image[..., :ind2interp], new_slice, np_image[...,
                                                                     ind2interp:]),
                    axis=2)
                warnings.warn(
                    "One slice is missing, we replaced it by linear interpolation")
            else:
                # if more than one slice are missing
                raise RuntimeError('Multiple slices are missing')

        image_position_patient = [float(k) for k in slices[0].ImagePositionPatient]

        sitk_image = get_sitk_volume_from_np(np_image, pixel_spacing,
                                             image_position_patient)

        #correct_patient_name(str(slices[0].PatientName))
        output_filepath = join(
                output_folder,
                patientID + '_' + filename + '_' +
                modality.lower() + extension)
        sitk_writer = sitk.ImageFileWriter()
        sitk_writer.SetImageIO('NiftiImageIO')
        sitk_writer.SetFileName(output_filepath)
        sitk_writer.Execute(sitk_image)
    if modality == 'RTSTRUCT':
        rtstruct_file = input_filepaths
        for rt_file in rtstruct_file:

            print("Converting RTSTRUCT image")

            masks = get_masks(rt_file,
                            labels=labels_rtstruct,
                            image_position_patient=image_position_patient,
                            axial_positions=axial_positions,
                            pixel_spacing=pixel_spacing,
                            shape=np_image.shape,
                            dtype=dtype_image)
            for label, np_mask in masks: #label.lower()
                print("Labels: ",label)
                
                #output_filepath = join(output_folder,patientID + '_' +modality.lower() + extension)
                output_filepath = join(output_folder,filename+'_'+modality.lower() + extension)
                sitk_writer = sitk.ImageFileWriter()
                sitk_writer.SetImageIO('NiftiImageIO')                
                output_filepath_mask = output_filepath.split('.')[0] + '_' + label + extension
                print(output_filepath_mask)
                
                
                
                sitk_mask = get_sitk_volume_from_np(np_mask, pixel_spacing,
                                                    image_position_patient)
                
                sitk_writer.SetFileName(output_filepath_mask)
                sitk_writer.Execute(sitk_mask)

    #return np.transpose(np_image,(1, 0, 2)), pixel_spacing, image_position_patient
    return np.transpose(np_image,(1, 0, 2)), pixel_spacing, image_position_patient,axial_positions


def get_sitk_volume_from_np(np_image, pixel_spacing, image_position_patient):
    trans = (2, 0, 1)
    sitk_image = sitk.GetImageFromArray(np.transpose(np_image, trans))
    sitk_image.SetSpacing(pixel_spacing)
    sitk_image.SetOrigin(image_position_patient)
    return sitk_image


def is_approx_equal(x, y, tolerance=0.05):
    return abs(x - y) <= tolerance


class MissingWeightException(RuntimeError):
    pass


def correct_patient_name(patient_name):
    output = patient_name.replace("HN-CHUS-", "CHUS")
    output = output.replace("HN-CHUM-", "CHUM")
    output = output.replace("HN-HGJ-", "CHGJ")
    output = output.replace("HN-HMR-", "CHMR")
    output = output.replace("HN-CHUV-", "CHUV")
    return output


def get_masks(rtstruct_file,
              labels, #=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt'],
              image_position_patient=None,
              axial_positions=None,
              pixel_spacing=None,
              shape=None,
              dtype=np.int8):
    print('in get_mask')
    contours = read_structure(rtstruct_file, labels=labels)
    return get_mask_from_contour(contours,
                                 image_position_patient,
                                 axial_positions,
                                 pixel_spacing,
                                 shape,
                                 dtype=dtype)

#labels=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt']

def read_structure(rtstruct_file, labels):#=['gtvp', 'gtv tumor', 'gtv_pt','gtv-tumor','gtv-pt','gtvprim_tumor','gtv','gtvpt','gtv pt','gtv_tumor','gtvprimairetumor','gtv primaire tumor','gtv  primaire tumor','gtvtumor','gtv_p','gtv pet+ct','gtv pet-ct']):
    structure = pdcm.read_file(rtstruct_file)
    contours = []
    for i, roi_seq in enumerate(structure.StructureSetROISequence):
        contour = {}
        print(roi_seq.ROIName)
        for label in labels:
            
            if roi_seq.ROIName.lower() == label.lower():
                contour['color'] = structure.ROIContourSequence[
                    i].ROIDisplayColor
                contour['number'] = structure.ROIContourSequence[
                    i].ReferencedROINumber
                contour['name'] = roi_seq.ROIName
                assert contour['number'] == roi_seq.ROINumber
                contour['contours'] = [
                    s.ContourData
                    for s in structure.ROIContourSequence[i].ContourSequence
                ]
                contours.append(contour)

    return contours

def read_headers(rtstruct_file, labels=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt',  'GTVp']):
    structure = pdcm.read_file(rtstruct_file)
    gtv_names = []
    try:
        for i, roi_seq in enumerate(structure.StructureSetROISequence):
            
            if  'gtv' in roi_seq.ROIName.lower() or 'primaire' in roi_seq.ROIName.lower() or 'lk' in roi_seq.ROIName.lower() or 'klie' in roi_seq.ROIName.lower() or 'ln' in roi_seq.ROIName.lower() or 'ctv' in roi_seq.ROIName.lower():

                gtv_names.append(roi_seq.ROIName)
            #gtv_names = gtv_names + [j for j in list(roi_seq.ROIName) if 'GTV' in j or 'gtv' in j]

        return list(set(gtv_names))
    except:
        return list()

def choose_rtstruct(rtstruct_file):

    for name in rtstruct_file:

        print(name)

        structure = pdcm.read_file(name)

        try:
            for i, roi_seq in enumerate(structure.StructureSetROISequence):
                print(roi_seq.ROIName)
                if  'gtv' in roi_seq.ROIName.lower():
                #if  ('gtvpt' in roi_seq.ROIName.lower()) or ('gtv pt' in roi_seq.ROIName.lower()) or ('gtv-pt' in roi_seq.ROIName.lower()) or ('gtv  primaire tumor' in roi_seq.ROIName.lower()):
                    print('Found name ' + roi_seq.ROIName)
                    return [name]

            for i, roi_seq in enumerate(structure.StructureSetROISequence):
                print(roi_seq.ROIName)
                if  'primaire' in roi_seq.ROIName.lower():
                #if  ('gtvpt' in roi_seq.ROIName.lower()) or ('gtv pt' in roi_seq.ROIName.lower()) or ('gtv-pt' in roi_seq.ROIName.lower()) or ('gtv  primaire tumor' in roi_seq.ROIName.lower()):
                    print('Found name ' + roi_seq.ROIName)
                    return [name]
        except:
            print('Problem with RTSTRUCT')
            continue
            
    return list()

        #print(roi_seq.ROIName)
        #for label in labels:
            
            #print(roi_seq.ROIName)


def get_mask_from_contour(contours,
                          image_position_patient,
                          axial_positions,
                          pixel_spacing,
                          shape,
                          dtype=np.uint8):

    print('get_mask_from_c')
    z = np.asarray(axial_positions)
    pos_r = image_position_patient[1]
    spacing_r = pixel_spacing[1]
    pos_c = image_position_patient[0]
    spacing_c = pixel_spacing[0]

    output = []
    for con in contours:
        try:
            mask = np.zeros(shape, dtype=dtype)
            for current in con['contours']:
                nodes = np.array(current).reshape((-1, 3))
                assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
                z_index = np.where((nodes[0, 2] - 0.001 < z)
                                & (z < nodes[0, 2] + 0.001))[0][0]
                r = (nodes[:, 1] - pos_r) / spacing_r
                c = (nodes[:, 0] - pos_c) / spacing_c
                rr, cc = polygon(r, c)
                if len(rr) > 0 and len(cc) > 0:
                    if np.max(rr) > 512 or np.max(cc) > 512:
                        raise Exception("The RTSTRUCT file is compromised")

                mask[rr, cc, z_index] = 1
            output.append((con['name'], mask))
        except Exception as e:
            print(e)
            continue
    return output


def get_physical_values_ct(slices, dtype=np.float32):
    image = list()
    for s in slices:
        image.append(
            float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept))
    return np.stack(image, axis=-1).astype(dtype)


def get_physical_values_pt_old(slices):
    units = slices[0].Units
    s = slices[0]
    if units == 'BQML':
        datetime_acquisition = datetime.strptime(
            s[Tag(0x00080022)].value + s[Tag(0x00080032)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        datetime_serie = datetime.strptime(
            s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        if datetime_serie < datetime_acquisition and datetime_serie > datetime(
                1950, 1, 1):
            pass
        else:
            pass
        return get_suv_from_bqml(slices)
    elif units == 'CNTS':
        return get_suv_philips(slices)
    else:
        raise ValueError('The {} units is not handled'.format(units))

def get_physical_values_pt_new(slices, patient_weight, dtype=np.float32):
    s = slices[0]
    units = s.Units

    if units == 'BQML':

        start_time_str = s.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime
        start_time = time(int(start_time_str[0:2]),
                            int(start_time_str[2:4]),
                            int(start_time_str[4:6]))
        
        scan_time = datetime.strptime(
                    s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
                    "%Y%m%d%H%M%S")
        start_datetime = datetime.combine(scan_time.date(), start_time)
        
        decay_time = (scan_time-start_datetime).total_seconds()

        return get_suv_from_bqml_new(slices,
                                 decay_time,
                                 patient_weight,
                                 dtype=dtype)

def get_suv_from_bqml_new(slices, decay_time, patient_weight, dtype=np.float32):
    # Get SUV from raw PET
    image = list()
    for s in slices:
        pet = float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept)
        half_life = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
        total_dose = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
        decay = 2**(-decay_time / half_life)
        actual_activity = total_dose * decay

        im = pet * patient_weight * 1000 / actual_activity
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


def get_physical_values_pt(slices, patient_weight, dtype=np.float32):
    s = slices[0]
    units = s.Units
    if units == 'BQML':

        acquisition_datetime = datetime.strptime(
            s[Tag(0x00080022)].value + s[Tag(0x00080032)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        serie_datetime = datetime.strptime(
            s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
            "%Y%m%d%H%M%S")

        try:
            if (serie_datetime <= acquisition_datetime) and (
                    serie_datetime > datetime(1950, 1, 1)):
                scan_datetime = serie_datetime
            else:
                scan_datetime_value = s[Tag(0x0009100d)].value
                if isinstance(scan_datetime_value, bytes):
                    scan_datetime_str = scan_datetime_value.decode(
                        "utf-8").split('.')[0]
                elif isinstance(scan_datetime_value, str):
                    scan_datetime_str = scan_datetime_value.split('.')[0]
                else:
                    raise ValueError(
                        "The value of scandatetime is not handled")
                scan_datetime = datetime.strptime(scan_datetime_str,
                                                  "%Y%m%d%H%M%S")

            start_time_str = s.RadiopharmaceuticalInformationSequence[
                0].RadiopharmaceuticalStartTime
            start_time = time(int(start_time_str[0:2]),
                              int(start_time_str[2:4]),
                              int(start_time_str[4:6]))
            start_datetime = datetime.combine(scan_datetime.date(), start_time)
            decay_time = (scan_datetime - start_datetime).total_seconds()
        except KeyError:
            warnings.warn("Estimation of time decay for SUV"
                          " computation from average parameters")
            decay_time = 1.75 * 3600  # From Martin's code
        return get_suv_from_bqml(slices,
                                 decay_time,
                                 patient_weight,
                                 dtype=dtype)

    elif units == 'CNTS':
        return get_suv_philips(slices, dtype=dtype)
    else:
        raise ValueError('The {} units is not handled'.format(units))



def get_suv_philips(slices, dtype=np.float32):
    image = list()
    suv_scale_factor_tag = Tag(0x70531000)
    for s in slices:
        im = (float(s.RescaleSlope) * s.pixel_array +
              float(s.RescaleIntercept)) * float(s[suv_scale_factor_tag].value)
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


def get_suv_from_bqml(slices, decay_time, patient_weight, dtype=np.float32):
    # Get SUV from raw PET
    image = list()
    for s in slices:
        pet = float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept)
        half_life = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
        total_dose = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
        decay = 2**(-decay_time / half_life)
        actual_activity = total_dose * decay

        im = pet * patient_weight * 1000 / actual_activity
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


In [ ]:
def saveNifti(np_image,path_to_save,pixdims):
    converted_array = np.array(np_image, dtype=np.float32)
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nifti_file.header['pixdim'] = pixdims
    nib.save(nifti_file, path_to_save) 
    return 0

In [ ]:
def convert_dicom_to_nifty(input_filepaths,patientID,
                           output_folder,
                           labels_rtstruct,extension,filename=filename,
                           pixel_spacing = None, image_position_patient=None,axial_positions=None,
                           np_image=None,dtype_image=np.float32,dtype_mask=np.uint8,):

    slices = [dicom.read_file(dcm,force=True) for dcm in filesList]
    modality = slices[0].Modality
    print(modality)

    
    if modality == 'CT' or modality=='PT':
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

        # Check if all the slices come from the same serie
        same_serie_uid = True
        serie_uid = slices[0].SeriesInstanceUID
        for s in slices:
            same_serie_uid *= serie_uid == s.SeriesInstanceUID

        if not same_serie_uid:
            raise RuntimeError('A slice comes from another serie')

        axial_positions = np.asarray([k.ImagePositionPatient[2] for k in slices])

        # Compute redundant slice positions
        ind2rm = [
            ind for ind in range(len(axial_positions))
            if axial_positions[ind] == axial_positions[ind - 1]
        ]

        # Check if there is redundancy in slice positions and remove them
        if len(ind2rm) > 0:
            slices = [k for i, k in enumerate(slices) if i not in ind2rm]
            axial_positions = np.asarray(
                [k.ImagePositionPatient[2] for k in slices])
        else:
            print("len",len(ind2rm))

        slice_spacing = (slices[1].ImagePositionPatient[2] -
                         slices[0].ImagePositionPatient[2])

        pixel_spacing = np.asarray([
            slices[0].PixelSpacing[0],
            slices[0].PixelSpacing[1],
            slice_spacing,])
       
        if modality == 'CT' or modality == 'PT':
            np_image = get_physical_values_ct(slices, dtype=dtype_image)
        else:
            raise ValueError('The modality {} is not supported'.format(modality))


        position_final_slice = (
            len(slices) - 1) * slice_spacing + slices[0].ImagePositionPatient[2]

        # Test whether some slices are missing
        if not is_approx_equal(position_final_slice,
                               float(slices[-1].ImagePositionPatient[2])):
            if (position_final_slice - axial_positions[-1]) / slice_spacing < 1.5:
                # If only one slice is missing
                diff = np.asarray([
                    not is_approx_equal(
                        float(axial_positions[ind]) -
                        float(axial_positions[ind - 1]) - slice_spacing, 0)
                    for ind in range(1, len(axial_positions))
                ])
                ind2interp = int(np.where(diff)[0])
                new_slice = (np_image[:, :, ind2interp] +
                             np_image[:, :, ind2interp + 1]) * 0.5
                new_slice = new_slice[..., np.newaxis]
                np_image = np.concatenate(
                    (np_image[..., :ind2interp], new_slice, np_image[...,
                                                                     ind2interp:]),
                    axis=2)
                print("One slice is missing, we replaced it by linear interpolation")
            else:
                # if more than one slice are missing
                raise RuntimeError('Multiple slices are missing')
        else:
            print("is approx equal position _final")


        image_position_patient = [float(k) for k in slices[0].ImagePositionPatient]
        sitk_image = get_sitk_volume_from_np(np_image, pixel_spacing,
                                             image_position_patient)
        print("Converting",modality ,"image")
        print(pixel_spacing,image_position_patient)
        #_ = saveNifti(np_image,output_filepath,pixel_spacing)
        
        output_filepath = join(
            output_folder,
            #patientID + '_' + filename + '_' +
            filename + '_' +
            modality.lower() + extension)
        sitk_writer = sitk.ImageFileWriter()
        sitk_writer.SetImageIO('NiftiImageIO')
        sitk_writer.SetFileName(output_filepath)
        sitk_writer.Execute(sitk_image)

    if modality == 'RTSTRUCT':
        rtstruct_file = input_filepaths
        for rt_file in rtstruct_file:

            print("Converting RTSTRUCT image")

            masks = get_masks(rt_file,
                            labels=labels_rtstruct,
                            image_position_patient=image_position_patient,
                            axial_positions=axial_positions,
                            pixel_spacing=pixel_spacing,
                            shape=np_image.shape,
                            dtype=dtype_image)
            for label, np_mask in masks: #label.lower()
                print("Labels: ",label)
                
                #output_filepath = join(output_folder,patientID + '_' +modality.lower() + extension)
                output_filepath = join(output_folder,filename+'_'+modality.lower() + extension)
                sitk_writer = sitk.ImageFileWriter()
                sitk_writer.SetImageIO('NiftiImageIO')
                
                output_filepath_mask = output_filepath.split(
                    '.')[0] + '_' + label + extension
                print(output_filepath_mask)
                
                
                
                sitk_mask = get_sitk_volume_from_np(np_mask, pixel_spacing,
                                                    image_position_patient)
                
                sitk_writer.SetFileName(output_filepath_mask)
                sitk_writer.Execute(sitk_mask)

    return np.transpose(np_image,
                        (1, 0, 2)), pixel_spacing, image_position_patient,axial_positions


def get_sitk_volume_from_np(np_image, pixel_spacing, image_position_patient):
    trans = (2, 0, 1)
    sitk_image = sitk.GetImageFromArray(np.transpose(np_image, trans))
    sitk_image.SetSpacing(pixel_spacing)
    sitk_image.SetOrigin(image_position_patient)
    return sitk_image


def is_approx_equal(x, y, tolerance=0.05):
    print("TOLERANCE (0.05)",abs(x - y))
    return abs(x - y) <= tolerance


def get_masks(rtstruct_file,
              labels,
              image_position_patient=None,
              axial_positions=None,
              pixel_spacing=None,
              shape=None,
              dtype=np.int8):
    contours = read_structure(rtstruct_file, labels=labels)
    return get_mask_from_contour(contours,
                                 image_position_patient,
                                 axial_positions,
                                 pixel_spacing,
                                 shape,
                                 dtype=dtype)


def read_structure(rtstruct_file, labels):
    structure = dicom.read_file(rtstruct_file)
    contours = []
    for i, roi_seq in enumerate(structure.StructureSetROISequence):
        print("roi",roi_seq.ROIName)
        contour = {}
        for label in labels:
            if roi_seq.ROIName == label:
                if True:#roi_seq.ROIName.lower() == label.lower():
                    contour['color'] = structure.ROIContourSequence[
                        i].ROIDisplayColor
                    contour['number'] = structure.ROIContourSequence[
                        i].ReferencedROINumber
                    contour['name'] = roi_seq.ROIName
                    assert contour['number'] == roi_seq.ROINumber
                    contour['contours'] = [
                        s.ContourData
                        for s in structure.ROIContourSequence[i].ContourSequence
                    ]
                    contours.append(contour)

    return contours

def get_mask_from_contour(contours,
                          image_position_patient,
                          axial_positions,
                          pixel_spacing,
                          shape,
                          dtype=np.uint8):

    z = np.asarray(axial_positions)
    pos_r = image_position_patient[1]
    spacing_r = pixel_spacing[1]
    pos_c = image_position_patient[0]
    spacing_c = pixel_spacing[0]

    output = []
    for con in contours:
        try:
            mask = np.zeros(shape, dtype=dtype)
            for current in con['contours']:
                nodes = np.array(current).reshape((-1, 3))
                assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
                z_index = np.where((nodes[0, 2] - 0.001 < z)
                                & (z < nodes[0, 2] + 0.001))[0][0]
                r = (nodes[:, 1] - pos_r) / spacing_r
                c = (nodes[:, 0] - pos_c) / spacing_c
                rr, cc = polygon(r, c)
                if len(rr) > 0 and len(cc) > 0:
                    if np.max(rr) > 512 or np.max(cc) > 512:
                        raise Exception("The RTSTRUCT file is compromised")

                mask[rr, cc, z_index] = 1
            output.append((con['name'], mask))
        except Exception as e:
            print(e)
            continue
    return output


def get_physical_values_ct(slices, dtype=np.float32):
    image = list()
    for s in slices:
        image.append(
            float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept))
    return np.stack(image, axis=-1).astype(dtype)

def get_last_dir(path):

    path=path.replace(os.path.basename(path),"")[:-1]

    return path

def create_dir(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)